In [2]:
#these are the required libraries for this part
!pip install opencv-python
!pip install fiftyone
!pip install torchvision
!pip install torch


from google.colab import files
uploaded = files.upload()

Saving tracking_assignment1.mp4 to tracking_assignment1 (1).mp4


In [13]:
import cv2
import os

#this is the same code from the last part to extract the frames from the uploaded video
def extract_frames(video_path, output_folder, num_frames=1000):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // num_frames)
    frame_count = 0
    image_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0 and image_count < num_frames:
            image_name = f"frame_{image_count:04d}.jpg"
            image_path = os.path.join(output_folder, image_name)
            cv2.imwrite(image_path, frame)
            image_count += 1
        frame_count += 1

    cap.release()
    print(f"Successfully extracted {image_count} frames to '{output_folder}'.")


video_file = next(iter(uploaded.keys()))
output_directory = "video_frames"
extract_frames(video_file, output_directory, num_frames=1000)

Successfully extracted 1000 frames to 'video_frames'.


In [19]:
import fiftyone as fo
import torch
import torchvision
from torch.utils.data import Dataset
import os

class VideoFramesDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_paths = sorted([os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith('.jpg')])

    def __len__(self):
        return len(self.image_paths)

def load_and_visualize_dataset(image_folder_path, dataset_name="video_frames_dataset"):
    # Delete the dataset if it already exists
    if dataset_name in fo.list_datasets():
        fo.delete_dataset(dataset_name)

    dataset = fo.Dataset(dataset_name)
    image_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith('.jpg')]
    samples = [fo.Sample(filepath=path) for path in image_paths]
    dataset.add_samples(samples)
    print(dataset)
    session = fo.launch_app(dataset)
    print("FiftyOne App is running!! Yay!!!!")

load_and_visualize_dataset(output_directory)

 100% |███████████████| 1000/1000 [214.8ms elapsed, 0s remaining, 4.7K samples/s]     


INFO:eta.core.utils: 100% |███████████████| 1000/1000 [214.8ms elapsed, 0s remaining, 4.7K samples/s]     


Name:        video_frames_dataset
Media type:  image
Num samples: 1000
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField


FiftyOne App is running!! Yay!!!!
